In [9]:
import torch
import time
import numpy as np


def benchmark_activation(activation_fn, name, n_iterations=10000):
    
    x = torch.randn(1024, 1024, device='cpu')
    x.requires_grad = True

    
    for __ in range(100):
        y = activation_fn(x)
        y.sum().backward()
    
    
    start = time.time()
    for _ in range(n_iterations):
        y = activation_fn(x)
        torch.cuda.synchronize()
    forward_time = (time.time() - start) / n_iterations
    
    
    start = time.time()
    for _ in range(n_iterations):
        x.grad = None
        y = activation_fn(x)
        y.sum().backward()
        torch.cuda.synchronize()
    total_time = (time.time() - start) / n_iterations
    
    backward_time = total_time - forward_time
    
    return forward_time, backward_time, total_time

relu = torch.nn.ReLU()
silu = torch.nn.SiLU()
gelu = torch.nn.GELU()

def mish(x):
    return x * torch.tanh(torch.nn.functional.softplus(x))